## Práctica de Inteligencia Artificial

La práctica consiste en entrenar un algoritmo para que calcule el precio de un jugador de fútbol del juego FIFA a partir de un dataset del mismo juego.

### Herramientas

Las herramientas que vamos a usar son las librerías Pandas, Numpy y Scikit Learn de Python.

In [1]:
import os

#separamos el dataframe en train y test
from sklearn.model_selection import train_test_split
#vamos a usar un modelo lineal
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import pandas as pd
import numpy as np

### Leer y analizar los datos
Ahora podemos leer el dataframe.

In [2]:
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))
pd.set_option('display.max_columns', None)
df

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

Ahora tenemos que buscar las columnas que no tienen relevancia y eliminarlas.

In [3]:
df = df.iloc[:, 2:]
df = df.drop('Photo', 1)
df = df.drop('Flag', 1)
df = df.drop('Club Logo', 1)
df = df.drop('Joined', 1)
df = df.drop('Loaned From', 1)
df = df.drop('Contract Valid Until', 1)
df = df.drop('Real Face', 1)
df = df.drop('Jersey Number', 1)
df = df.drop('Release Clause', 1)
df = df.drop('Name', 1)

Hay algunas columnas en las cuales en vez de un valor hay un NaN. Por ejemplo, algunos jugadores no tienen un club asignado. Podemos borrarlos ya que son pocos. También podemos borrar a los porteros que no tienen asignadas las características LS, ST, etc.

In [4]:
df = df.dropna(subset = ['Club'])
df = df.dropna(subset = ['LS'])

Como vemos, las columnas que por lógica deberían contener un número (Value, Wage), contienen una cadena de caracteres.
Tenemos que pasar estos valores a entero con una función.

In [5]:
def value_to_float(x):
    x = x.replace('€', '')
    ret_val = 0.0
    if type(x) == float or type(x) == int:
        ret_val = x
    if 'K' in x:
        if len(x) > 1:
            ret_val = float(x.replace('K', ''))
        ret_val = ret_val *1000
    if 'M' in x:
        if len(x) > 1:
            ret_val = float(x.replace('M', ''))
        ret_val = ret_val * 1000000.0
    return ret_val

In [6]:
df["Value"] = df["Value"].apply(value_to_float)
df["Wage"] = df["Wage"].apply(value_to_float)

Ahora convertimos la columna Weight en un entero.

In [7]:
def weight_to_float(x):
    x = x.replace("lbs", "")
    ret_val = float(x)
    return ret_val

In [8]:
df["Weight"] = df["Weight"].apply(weight_to_float)

Hacemos lo mismo con la columna Height

In [9]:
def height_to_float(x):
    x = x.replace("'", ".")
    ret_val = float(x)
    return ret_val
    

In [10]:
df["Height"] = df["Height"].apply(height_to_float)

Ahora podemos convertir las columnas Nationality, Club, Preferred Foot, Work Rate, Body Type y Position en una estructura hot-encoding.

In [11]:
nat = df.pop("Nationality")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(nat, prefix='nat').reset_index(drop=True)], axis = 1, sort = False)

clb = df.pop("Club")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(clb, prefix='clb').reset_index(drop=True)], axis = 1, sort = False)

preffoot = df.pop("Preferred Foot")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(preffoot, prefix='preffoot').reset_index(drop=True)], axis = 1, sort = False)

wrate = df.pop("Work Rate")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(wrate, prefix='wrate').reset_index(drop=True)], axis = 1, sort = False)

btype = df.pop("Body Type")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(btype, prefix='btype').reset_index(drop=True)], axis = 1, sort = False)

pos = df.pop("Position")
df = pd.concat([df.reset_index(drop=True), pd.get_dummies(pos, prefix='pos').reset_index(drop=True)], axis = 1, sort = False)

Ahora podemos convertir las columnas con las características en formato numérico

In [12]:
def car_to_float(x):
    firstSum =  x[:2]
    secondSum =  x[-1]
    ret_val = float(firstSum) + float(secondSum)
    return ret_val


In [13]:
df["LS"] = df["LS"].apply(car_to_float)
df["ST"] = df["ST"].apply(car_to_float)
df["RS"] = df["RS"].apply(car_to_float)
df["LW"] = df["LW"].apply(car_to_float)
df["LF"] = df["LF"].apply(car_to_float)
df["CF"] = df["CF"].apply(car_to_float)
df["RF"] = df["RF"].apply(car_to_float)
df["RW"] = df["RW"].apply(car_to_float)
df["LAM"] = df["LAM"].apply(car_to_float)
df["CAM"] = df["CAM"].apply(car_to_float)
df["RAM"] = df["RAM"].apply(car_to_float)
df["LM"] = df["LM"].apply(car_to_float)
df["LCM"] = df["LCM"].apply(car_to_float)
df["CM"] = df["CM"].apply(car_to_float)
df["RCM"] = df["RCM"].apply(car_to_float)
df["RM"] = df["RM"].apply(car_to_float)
df["LWB"] = df["LWB"].apply(car_to_float)
df["LDM"] = df["LDM"].apply(car_to_float)
df["CDM"] = df["CDM"].apply(car_to_float)
df["RDM"] = df["RDM"].apply(car_to_float)
df["RWB"] = df["RWB"].apply(car_to_float)
df["LB"] = df["LB"].apply(car_to_float)
df["LCB"] = df["LCB"].apply(car_to_float)
df["CB"] = df["CB"].apply(car_to_float)
df["RCB"] = df["RCB"].apply(car_to_float)
df["RB"] = df["RB"].apply(car_to_float)
df


,Age,Overall,Potential,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverkusen,clb_Beijing Renhe FC,clb_Beij

### Predecir el precio
Ahora tenemos preparado el dataframe y podemos separar la columna que queremos predecir.

In [14]:
val = df.pop("Value")
#df = df.drop("Value", 1)
df

,Age,Overall,Potential,Wage,Special,International Reputation,Weak Foot,Skill Moves,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,nat_Afghanistan,nat_Albania,nat_Algeria,nat_Andorra,nat_Angola,nat_Antigua & Barbuda,nat_Argentina,nat_Armenia,nat_Australia,nat_Austria,nat_Azerbaijan,nat_Barbados,nat_Belarus,nat_Belgium,nat_Belize,nat_Benin,nat_Bermuda,nat_Bolivia,nat_Bosnia Herzegovina,nat_Botswana,nat_Brazil,nat_Bulgaria,nat_Burkina Faso,nat_Burundi,nat_Cameroon,nat_Canada,nat_Cape Verde,nat_Central African Rep.,nat_Chad,nat_Chile,nat_China PR,nat_Colombia,nat_Comoros,nat_Congo,nat_Costa Rica,nat_Croatia,nat_Cuba,nat_Curacao,nat_Cyprus,nat_Czech Republic,nat_DR Congo,nat_Denmark,nat_Dominican Republic,nat_Ecuador,nat_Egypt,nat_El Salvador,nat_England,nat_Equatorial Guinea,nat_Eritrea,nat_Estonia,nat_Ethiopia,nat_FYR Macedonia,nat_Faroe Islands,nat_Fiji,nat_Finland,nat_France,nat_Gabon,nat_Gambia,nat_Georgia,nat_Germany,nat_Ghana,nat_Greece,nat_Grenada,nat_Guam,nat_Guatemala,nat_Guinea,nat_Guinea Bissau,nat_Guyana,nat_Haiti,nat_Honduras,nat_Hong Kong,nat_Hungary,nat_Iceland,nat_Indonesia,nat_Iran,nat_Iraq,nat_Israel,nat_Italy,nat_Ivory Coast,nat_Jamaica,nat_Japan,nat_Jordan,nat_Kazakhstan,nat_Kenya,nat_Korea DPR,nat_Korea Republic,nat_Kosovo,nat_Kuwait,nat_Latvia,nat_Lebanon,nat_Liberia,nat_Libya,nat_Liechtenstein,nat_Lithuania,nat_Luxembourg,nat_Madagascar,nat_Mali,nat_Malta,nat_Mauritania,nat_Mauritius,nat_Mexico,nat_Moldova,nat_Montenegro,nat_Montserrat,nat_Morocco,nat_Mozambique,nat_Namibia,nat_Netherlands,nat_New Caledonia,nat_New Zealand,nat_Nicaragua,nat_Niger,nat_Nigeria,nat_Northern Ireland,nat_Norway,nat_Palestine,nat_Panama,nat_Paraguay,nat_Peru,nat_Philippines,nat_Poland,nat_Portugal,nat_Puerto Rico,nat_Qatar,nat_Republic of Ireland,nat_Romania,nat_Russia,nat_Rwanda,nat_Saudi Arabia,nat_Scotland,nat_Senegal,nat_Serbia,nat_Sierra Leone,nat_Slovakia,nat_Slovenia,nat_South Africa,nat_South Sudan,nat_Spain,nat_St Kitts Nevis,nat_St Lucia,nat_Sudan,nat_Suriname,nat_Sweden,nat_Switzerland,nat_Syria,nat_São Tomé & Príncipe,nat_Tanzania,nat_Thailand,nat_Togo,nat_Trinidad & Tobago,nat_Tunisia,nat_Turkey,nat_Uganda,nat_Ukraine,nat_United Arab Emirates,nat_United States,nat_Uruguay,nat_Uzbekistan,nat_Venezuela,nat_Wales,nat_Zambia,nat_Zimbabwe,clb_ SSV Jahn Regensburg,clb_1. FC Heidenheim 1846,clb_1. FC Kaiserslautern,clb_1. FC Köln,clb_1. FC Magdeburg,clb_1. FC Nürnberg,clb_1. FC Union Berlin,clb_1. FSV Mainz 05,clb_AC Ajaccio,clb_AC Horsens,clb_AD Alcorcón,clb_ADO Den Haag,clb_AEK Athens,clb_AFC Wimbledon,clb_AIK,clb_AJ Auxerre,clb_AS Béziers,clb_AS Monaco,clb_AS Nancy Lorraine,clb_AS Saint-Étienne,clb_AZ Alkmaar,clb_Aalborg BK,clb_Aarhus GF,clb_Aberdeen,clb_Accrington Stanley,clb_Adelaide United,clb_Ajax,clb_Akhisar Belediyespor,clb_Al Ahli,clb_Al Batin,clb_Al Faisaly,clb_Al Fateh,clb_Al Fayha,clb_Al Hazem,clb_Al Hilal,clb_Al Ittihad,clb_Al Nassr,clb_Al Qadisiyah,clb_Al Raed,clb_Al Shabab,clb_Al Taawoun,clb_Al Wehda,clb_Alanyaspor,clb_Albacete BP,clb_Alianza Petrolera,clb_Amiens SC,clb_América FC (Minas Gerais),clb_América de Cali,clb_Angers SCO,clb_Antalyaspor,clb_Argentinos Juniors,clb_Arka Gdynia,clb_Arsenal,clb_Ascoli,clb_Aston Villa,clb_Atalanta,clb_Athletic Club de Bilbao,clb_Atiker Konyaspor,clb_Atlanta United,clb_Atlético Bucaramanga,clb_Atlético Huila,clb_Atlético Madrid,clb_Atlético Mineiro,clb_Atlético Nacional,clb_Atlético Paranaense,clb_Atlético Tucumán,clb_Audax Italiano,clb_BB Erzurumspor,clb_BK Häcken,clb_BSC Young Boys,clb_Bahia,clb_Barnsley,clb_Bayer 04 Leverkusen,clb_Beijing Renhe FC,clb_Beijing Si

Ya podemos dividir los datos en train/test y ejecutar el aprendizaje automático.

In [15]:
x_train, x_test, y_train, y_test = train_test_split(df, val, test_size = 0.3, random_state = 42)

In [16]:
len(x_train)

11148

In [17]:
reg = linear_model.LinearRegression().fit(x_train, y_train)

In [18]:
preds = reg.predict(x_test)

In [19]:
preds[0]

-107597.13025204837

In [20]:
y_test[0]

110500000.0

In [21]:
r2_score(preds, y_test)

0.799164988714544

Como podemos ver, el entrenamiento ha fallado. Vamos a ver si tenemos algun error en las columnas (NaN)

In [22]:
df.columns[df.isna().any()].tolist()

[]

Como no hay columnas con errores, quedan 2 variantes a ver: puede que el algoritmo de aprendizaje automático se haya equivocado debido a la gran variedad de las columnas y la gran dispersión de los valores o, sinó, podemos deducir que no se puede calcular el precio de un jugador debido a que no hay bastante relación entre el precio y las otras características.